## Implementacja wybranych metryk

### Metryka Levensteina

In [1]:
# can be applied to iterables of tokens instead of text as well
def levenstein(text1, text2):
    n1, n2 = len(text1), len(text2)
    A = [[None] * (n1+1) for _ in range(n2+1)]
    
    A[0][0] = (0, None)
    for i in range(1, n1+1):
        A[0][i] = (i, (0, i-1))
    for j in range(1, n2+1):
        A[j][0] = (j, (j-1, 0))
    
    for i in range(1, n1+1):
        for j in range(1, n2+1):
            add = 1
            if text1[i-1] == text2[j-1]:
                add = 0

            x = min(A[j-1][i][0]+1, A[j][i-1][0]+1, A[j-1][i-1][0]+add)
            if A[j-1][i][0] + 1 == x:
                A[j][i] = (x, (j-1, i))
            elif A[j][i-1][0] + 1 == x:
                A[j][i] = (x, (j, i-1))
            else:
                A[j][i] = (x, (j-1, i-1))

    return A[-1][-1][0]

### Metryka DICE

In [2]:
def dice(text1, text2):
    letters1 = set(text1)
    letters2 = set(text2)
    return 2 * len(letters1.intersection(text2)) / (len(letters1) + len(letters2))

### Metryka LCS

In [3]:
def lcs(text1, text2):
    n, m = len(text1), len(text2)
    
    A = [[None for _ in range(m + 1)] for _ in range(n+1)]
    for i in range(n+1):
        A[i][0] = 0
    for i in range(m+1):
        A[0][i] = 0
        
    for i in range(1, n+1):
        for j in range(1, m+1):
            if text1[i-1] == text2[j-1]:
                A[i][j] = A[i-1][j-1] + 1
            else:
                A[i][j] = max(A[i][j-1], A[i-1][j])
    
    return A[-1][-1]

## Jakość klasteryzacji - Rand index (nie mylić z losowym)

Wskaźnik liczony jako łączna liczba par które w obu wynikach są w tym samym klastrze lub w obu wynikach są w różnych klastrach, dzielona przez liczbę par.

In [4]:
def rand_index(result1, result2):
    n = len(result1)
    enumerator = 0
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            if (result1[i] == result1[j] and result2[i] == result2[j]) or \
               (result1[i] != result1[j] and result2[i] != result2[j]):
                enumerator += 1
    return enumerator / (n * (n-1))

## Stoplista

In [5]:
def get_stop_words(words, top=10):
    popularity = {}
    for w in words:
        popularity[w] = popularity.setdefault(w, 0) + 1
    
    words = sorted(filter(lambda x: len(x[0]) >= 3, popularity.items()), key=lambda x: x[1], reverse=True)
    return words[:min(len(words), top)]

In [6]:
with open("lines.txt", "r") as f:
    full = f.read().lower()
    words = full.split()

stopwords = get_stop_words(words, 100) # top 100 words, last word here has >100 ocurrences
# word and it's count
get_stop_words(words)

[('china', 934),
 ('logistics', 822),
 ('poland', 820),
 ('ltd', 781),
 ('ltd.', 555),
 ('ul.', 469),
 ('tel:', 406),
 ('limited', 387),
 ('fax:', 375),
 ('road', 372)]

## Klasteryzacja

Do klasteryzacji wykorzystano algorytm affinity propagation z scikit-learn. Dane po preprocessingu to lista słów, z których usunięto znaki interpunkcyjne. Zastosowanie takich metryk dla danych przed przerobieniem powoduje, że "tokenami" są poszczególne litery, a algorytm zajmuje bardzo dużo czasu. Gdy tokenami są całe słowa - zajmuje to mniej czasu. W niektórych przypadkach musimy zanegować uzyskaną "odległość" - gdy metryka maleje jak teksty są podobne.

In [7]:
def process(lns):
    lines = lns.copy()
    for i in range(len(lines)):
        lines[i] = lines[i].lower()
    punctuation = ".,;/()[]\n\t\"':-+"
    for i in range(len(lines)):
        for c in punctuation:
            lines[i] = lines[i].replace(c, "").strip()
    return lines

In [8]:
def remove_stopwords(lns, stopwords):
    lines = lns.copy()
    for i in range(len(lines)):
        lines[i] = lines[i].lower()
    for w in stopwords:
        lines[i] = lines[i].replace(f"{w[0]}", " ")
    return lines

In [9]:
with open("lines.txt", "r") as f:
    lines = f.readlines()
    processed = remove_stopwords(process(lines), stopwords)

In [10]:
import numpy as np
from sklearn.cluster import AffinityPropagation
from sklearn.metrics import rand_score

In [11]:
N = 200
nl = [line.split() for line in remove_stopwords(process(lines), stopwords)[:N]]
# nl = lines[:N]

In [12]:
def get_labels(data, metric, mul=1):
    aff_matrix = mul * np.array([[metric(w1, w2) for w2 in data] for w1 in data])
    affprop = AffinityPropagation(affinity="precomputed", damping=0.5)
    affprop.fit(aff_matrix)
    return affprop.labels_[:]

In [13]:
def print_clusters(labels):
    for j in range(max(labels)+1):
        for i in range(len(labels)):
            if labels[i] == j:
                print(lines[i])
        print("-------------")

In [14]:
with open("clusters.txt", "r") as f:
    correct = f.read().split(sep="##########")
for i in range(len(correct)):
    correct[i] = correct[i].split(sep='\n')

In [15]:
true_labels = [None] * len(lines)
for i in range(len(lines)):
    for j in range(len(correct)):
        if lines[i][:-1] in correct[j]:
            true_labels[i] = j
            break

Niestety dla tak dobranych metryk (spośród których najszybszą jest metryka dice), czas policzenia macierzy odległości pomiędzy każdymi dwoma stringami jest znaczny. Udało się policzyć jedynie dla maksymalnie 1000 linii.

In [16]:
N = 50
nl = [line.split() for line in remove_stopwords(process(lines), stopwords)[:N]]
lev_labels = get_labels(nl, levenstein, -1)
dice_labels = get_labels(nl, dice)
lcs_labels = get_labels(nl, lcs)

In [17]:
print(rand_index(lev_labels, true_labels))
print(rand_index(dice_labels, true_labels))
print(rand_index(lcs_labels, true_labels))

0.9028571428571428
0.9183673469387755
0.9159183673469388


Dla tekstów bez preprocessingu pojawia się problem. "Tokenami" są poszczególne litery, a co za tym idzie jest ich znacznie więcej. Powoduje to znaczny wzrost czasu działania algorytmów.

In [18]:
N = 50
nl = [line for line in lines[:N]]
lev_labels = get_labels(nl, levenstein, -1)
dice_labels = get_labels(nl, dice)
lcs_labels = get_labels(nl, lcs)

In [19]:
print(rand_index(lev_labels, true_labels))
print(rand_index(dice_labels, true_labels))
print(rand_index(lcs_labels, true_labels))

0.8759183673469387
0.8579591836734693
0.8391836734693877


Porównując otrzymane wskaźniki między danymi które poddano preprocessingowi oraz nieobrobione, widzimy że lepsze wyniki otrzymano dla danych przetworzonych.
Warto sprawdzić też większą liczbę linii:

In [20]:
N = 100
nl = [line.split() for line in remove_stopwords(process(lines), stopwords)[:N]]
lev_labels = get_labels(nl, levenstein, -1)
dice_labels = get_labels(nl, dice)
lcs_labels = get_labels(nl, lcs)

C:\Users\Piotr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_affinity_propagation.py:236: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


In [21]:
print(rand_index(lev_labels, true_labels))
print(rand_index(dice_labels, true_labels))
print(rand_index(lcs_labels, true_labels))

0.9464646464646465
0.9577777777777777
0.9472727272727273


In [22]:
N = 100
nl = [line for line in lines[:N]]
lev_labels = get_labels(nl, levenstein, -1)
dice_labels = get_labels(nl, dice)
lcs_labels = get_labels(nl, lcs)

In [23]:
print(rand_index(lev_labels, true_labels))
print(rand_index(dice_labels, true_labels))
print(rand_index(lcs_labels, true_labels))

0.9547474747474748
0.9216161616161617
0.9329292929292929


Warto zauważyć że zastosowana metryka okazała się nie być najlepszym wyborem w tej sytuacji. Zwiększając liczbę elementów które klasteryzujemy, rośnie wartość metryki. Wynika to z faktu, że większość par elementów należy do różnych klastrów i pary te, które możemy uznać za "true negative" dominują nad parami pozytywnymi.

In [24]:
N = 500
nl = [line.split() for line in remove_stopwords(process(lines), stopwords)[:N]]
lev_labels = get_labels(nl, levenstein, -1)
dice_labels = get_labels(nl, dice)
lcs_labels = get_labels(nl, lcs)

C:\Users\Piotr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_affinity_propagation.py:236: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


In [25]:
print(rand_index(lev_labels, true_labels))
print(rand_index(dice_labels, true_labels))
print(rand_index(lcs_labels, true_labels))

0.9771703406813628
0.9878877755511022
0.9870140280561123


Dla linii poddanym preprocessingowi jesteśmy w stanie sklasyfikować w sensownym czasie znacznie większą liczbę linii.

## Czy dałoby się zrobić lepiej?

Próbując poprawić uzyskane wyniki, należałoby większą uwagę zwrócić na poprawę czasu działania klasteryzacji. Tak dobranymi metodami nie udało się sklasyfikować wszystkich linii (z obserwacji czas klasyfikacji linii poddanych preprocessingowi całego pliku wynosiłby kilka/kilkanaście godzin).<br>
Aby zwiększyć dokładność dobrym pomysłem mogłoby okazać się wyodrębnianie charakterystycznych elementów takich jak nr telefonu, fax, adres za pomocą specjalnie przygotowanych wyrażeń regularnych. Tak uzyskane dane mogłyby ze znacznie większą pewnością stwierdzić, czy dane elementy powinny należeć do jednego klastra.